In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os import path
%matplotlib inline

In [2]:
# Read data
data = pd.read_csv('Task4DataSet/02_train.csv')

In [3]:
test_data = pd.read_csv('Task4DataSet/02_test.csv')

In [4]:
data.insert(loc=0,column='x_0_1',value=np.ones(len(data))) 
data = data.dropna(axis=1, how='all')
test_data = data.dropna(axis=1, how='all')
test_data.insert(loc=0,column='1',value=np.ones(len(test_data))) 

In [5]:
def normalize(data):
    data = data.T
    data = np.array(data)
    mean_std = {'mean': [],
                'std': [] }
    normalized_data = []
    for values in data:
        normalized_data.append((values - np.mean(values)) /np.std(values))
        mean_std['mean'].append(np.mean(values))
        mean_std['std'].append(np.std(values))
    
    normalized_data = pd.DataFrame(normalized_data)
    return normalized_data.T, mean_std

        
        

In [6]:
# def normalize_with_mean_and_std(data, mean_std_dict):
#     data = data.T
#     data = np.array(data)
#     print(data.shape, len(mean_std_dict['mean']))
#     normalized_data = []
#     for i,values in enumerate(data):
#         normalized_data.append( (values - mean_std_dict['mean'][i]) / mean_std_dict['std'][i])
    
#     normalized_data = pd.DataFrame(normalized_data)
#     return normalized_data.T
    

In [7]:
data , std= normalize(data)

/anaconda3/envs/MasterThesisEnv/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [8]:
len(std['mean'])

221

In [9]:
# test_data = normalize_with_mean_and_std(test_data, std)

In [10]:
data = data.dropna(axis=1, how='all')
test_data = data.dropna(axis=1, how='all')
test_data.insert(loc=0,column='1',value=np.ones(len(test_data))) 

In [ ]:
print(test_data.shape)

In [ ]:
print(data.shape)

In [ ]:
data.head()

In [ ]:
data.head(10)

In [ ]:
X = data.iloc[:, :-1]
Y = data.iloc[:, -1]

In [ ]:
X_test = test_data.iloc[:, :-1]
Y_test = test_data.iloc[:,-1]

In [ ]:
# Generating Coefficients
B = np.zeros(len(X.columns))
print(B)
X = np.array(X)
Y = np.array(Y)
print(X.dtype)

In [ ]:
# HyperParameters 
learning_rate = 0.01
regularization_parameter = 1e-15 # 0.01

In [ ]:
# Defining Cost function
cost_function = lambda X,Y,B: np.sum(np.square(X.dot(B) - Y)) / (2.0 * len(Y))

In [ ]:
initial_cost = cost_function(X,Y,B)
print(initial_cost)

In [ ]:
# Writing Gradient Descent with L2 Ridge Regularization

def gradient_descent(X, Y, B, learning_rate, epochs,regularization_parameter):
    
    cost_history = []
    size = len(Y)
    for i in range(epochs):
        h = X.dot(B)
        loss = h - Y  - ( regularization_parameter / (2 * size) ) * np.sum(np.square(B))
        gradient = X.T.dot(loss) / size
        B = B - learning_rate * gradient# Lasso Regression - regularization_parameter*np.sign(B)
        # B[B < 1e-16] = 0     when lasso regression
        cost = cost_function(X,Y,B)
        if i % 10 == 0:
            cost_history.append(cost)
  
    return B, cost_history
    

In [ ]:
X.shape, Y.shape, B.shape

In [ ]:
newB, cost_history = gradient_descent(X, Y, B, learning_rate, 1000, regularization_parameter)

In [ ]:
a = np.linspace(0,1000,100)
plt.plot(a, cost_history)
plt.xlabel('Epochs')
plt.ylabel('Cost')
plt.title('Cost Function with Epochs')
plt.show()

In [ ]:
def rmse(Y, Y_pred):
    s = np.array(Y - Y_pred)
    s = np.array(s**2)
    s = np.sum(s)/ len(Y)
    return s

In [ ]:
# Predict Function : 
def predict(X, B ,std):
    X = X.T
    new_X = []
    for i,column in enumerate(X):
        if i != 1:
            new_X.append( (column - std['mean'][i]) / std['std'][i] )
        
    new_X = np.array(new_X).T
    return new_X.dot(B)

In [ ]:
Y_pred =  predict(X_test, newB, std )

In [ ]:
print(rmse(Y_test, Y_pred))

Alternate Method

In [ ]:
def method_of_regularized_least_squares(X_, Y_):
    # Regularized Least Squares
    L = 0.1
    CX = X_.T.dot(X_)
    B_ = (np.linalg.inv(CX + L * np.identity(len(CX)))).dot(X_.T.dot(Y_))

    return B_

In [ ]:
B = method_of_regularized_least_squares(X, Y)

In [ ]:
Y_pred = X_test.dot(B)

In [ ]:
print(rmse(Y_test, Y_pred))